In [3]:
import pandas as pd
from extractor import ExtractingPrompt
from LLM import LLM
from tqdm import tqdm

2025-03-06 15:10:44.325791: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 15:10:44.336616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741270244.350122 2671796 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741270244.354354 2671796 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 15:10:44.367519: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
df = pd.read_csv('/home/laajila/mima_newcode/clean_code/outputs/2025-02-05/14-14-20/output_20250205_144847.csv')

In [6]:
extractor = ExtractingPrompt()

In [7]:
phrase = "I feel nausea, and headache"

In [8]:
extractor.build_extraction_prompt("I feel nausea, and headache")

[{'role': 'system',
  'content': 'You are an AI assistant trained to extract structured symptoms from patient inputs based on the PRO-CTCAE dataset. You must identify and return only the symptoms mentioned, ensuring accuracy and completeness.'},
 {'role': 'user',
  'content': 'Identify and extract the symptoms described in the following patient statement:\n\n"I feel nausea, and headache"\n\nReturn only the symptoms, separated by commas if multiple, without any additional text, comments, or explanations.'}]

In [9]:
model = LLM(model_name="iRASC/BioLlama-Ko-8B", max_length=100)

/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded on device(s): {'': 0}


In [10]:
phrase = "I\'m so scared, I\'ve got these cracks at the corners of my mouth that won\'t go away, it\'s so painful and itchy, I\'m so worried I\'ll get an infection."

In [11]:
def build_extraction_prompt(dialogue: str) -> list[dict[str, str]]:
        
        """
        Builds a structured prompt instructing the LLM to extract symptoms from the provided dialogue.

        :param dialogue: The dialogue text from which to extract symptoms.
        :return: A structured list of messages in a format suitable for LLM-based chat models.
        """
        dialogue = dialogue.strip()  # Ensures clean input

        messages = [
            {
                "role": "system",
                "content": (
                    "You are a chatbot assistant trained to extract structured symptoms from patient inputs. "
                    "You must identify and return only the symptoms mentioned, strictly following the PRO-CTCAE formalism to ensure accuracy and completeness. "
                    "You must be concise and avoid repetitions."
                ),
            },
            {
                "role": "user",
                "content": (
                    "Identify and extract the symptoms described in the following patient statement: "
                    f"{dialogue}"
                    " Return only the symptoms, separated by commas if multiple, "
                    "without any additional text, comments, or explanations."
                ),
            },
        ]

        return messages

In [12]:
prompt = build_extraction_prompt(phrase)

In [13]:
prompt

[{'role': 'system',
  'content': 'You are a chatbot assistant trained to extract structured symptoms from patient inputs. You must identify and return only the symptoms mentioned, strictly following the PRO-CTCAE formalism to ensure accuracy and completeness. You must be concise and avoid repetitions.'},
 {'role': 'user',
  'content': "Identify and extract the symptoms described in the following patient statement: I'm so scared, I've got these cracks at the corners of my mouth that won't go away, it's so painful and itchy, I'm so worried I'll get an infection. Return only the symptoms, separated by commas if multiple, without any additional text, comments, or explanations."}]

In [14]:
model.generate_text(prompt)

' dry mouth, mouth ulcers, mouth pain, mouth itching, mouth cracks, infection, pain, itching, infection, dry mouth, mouth ulcers, mouth pain, mouth itching, mouth cracks, infection, pain, itching, infection, dry mouth, mouth ulcers, mouth pain, mouth itching, mouth cracks, infection, pain, itching, infection, dry mouth, mouth ulcers, mouth pain, mouth itching, mouth cracks, infection, pain, itching, infection, dry mouth, mouth'

In [3]:
import os
import random
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from LLM import LLM
from PromptBuilder import PromptBuilder
from metadata import language_registers, discussion_tones

import json
from hydra.utils import get_original_cwd
from omegaconf import DictConfig, OmegaConf

# Load configuration

# Load JSON configuration
with open("config.json", "r") as file:
    cfg = json.load(file)

In [27]:
# Initialize prompt builder and LLM with values from config.
prompt_builder = PromptBuilder()
model = LLM(model_name=cfg["model"]["model_name"], token=cfg["token"]["token_key"])

input_file = os.path.join(cfg["data"]["input_file"])

a = pd.read_excel(input_file, sheet_name="PRO")

index = (a[a['PRO-CTCAE PT'] == 'Pain and swelling at injection site']).index.values[0] + 1
symptoms = a.iloc[:index]['PRO-CTCAE PT'].values
symptoms = [s for s in symptoms if s != 'Other Symptoms']

/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded on device(s): {'': 0}


In [28]:
len(symptoms)

80

## Generate a multi-symptoms dataset 

In [ ]:
# Build a nested dictionary with the symptoms, descriptions, and meta values.
symptom_dict = {}
for symptom in symptoms:
    try:
        symptom_dict[symptom] = {}
        descriptions = a[a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute PT'].values[0].split(" || ")
        for description in descriptions:
            # Retrieve meta values for the description.
            meta_vals = a[a['PRO-CTCAE PT'] == description]['PRO-CTCAE Value PT'].values[0].split(" || ")
            symptom_dict[symptom][description] = meta_vals
    except Exception as e:
        print(f"Error processing {symptom}: {e}")

# Calculate the total number of iterations for progress tracking.
total_iterations = sum(
    len(meta_list) 
    for symptom in symptom_dict.values() 
    for meta_list in symptom.values()
)
pbar = tqdm(total=total_iterations, desc="Generating dialogues")

# Generate the dialogue data.
data = []
symptom_list = list(symptom_dict.keys())

for _ in range(100):  # Define how many sentences you want

    num_symptoms = np.random.randint(2, min(5, len(symptom_list) + 1))  # Random number of symptoms per sentence
    selected_symptoms = random.sample(symptom_list, num_symptoms)

    descriptions = []
    metas = []

    for symptom in selected_symptoms:
        
        if symptom != 'Other Symptoms':
            description = random.choice(list(symptom_dict[symptom].keys()))
            meta = random.choice(symptom_dict[symptom][description])
            descriptions.append(description)
            metas.append(meta)

    detail_level = np.random.choice(cfg["generation"]["detail_levels"])
    enumeration = np.random.choice([True, False], p=cfg["generation"]["enumeration_probability"])
    explicit_symptom = np.random.choice([True, False], p=cfg["generation"]["explicit_symptom_probability"])
    language_style = random.choice(language_registers)['name']
    tone = random.choice(discussion_tones)['name']
    spelling_errors = random.choice([True, False])

    prompt = prompt_builder.build_prompt(
        symptoms=selected_symptoms,
        description=descriptions,
        meta=metas,
        detail_level=detail_level,
        enumeration=enumeration,
        explicit_symptom=explicit_symptom,
        language_style=language_style,
        spelling_errors=spelling_errors,
        tone=tone
    )

    phrase_generated = model.generate_text(messages=prompt)

    data.append([
        phrase_generated, selected_symptoms, descriptions, metas, language_style, tone,
        detail_level, enumeration, explicit_symptom, spelling_errors
    ])

    pbar.update(1)

pbar.close()

# Create a DataFrame from the generated data.
df = pd.DataFrame(
    data,
    columns=[
        "Dialogue_Generated", "Symptoms", "Descriptions", "Meta",
        "Language_Style", "Tone", "Detail_Level", "Enumeration",
        "Explicit_Symptom", "Spelling_Errors"
    ]
)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

df.to_csv("Many_symptoms_generated_2.csv", index=False)

Generating dialogues:  12%|█▏        | 100/823 [12:19<1:29:06,  7.39s/it]
